In [1]:
import torch
import torchvision

from resnet_big import SupCEResNet

In [2]:
model = torchvision.models.resnet50(weights = None)

In [11]:
for name, param in list(model.named_parameters()):
    print(name)

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.conv3.weight
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.downsample.0.weight
layer1.0.downsample.1.weight
layer1.0.downsample.1.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.conv3.weight
layer1.1.bn3.weight
layer1.1.bn3.bias
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias
layer1.2.conv2.weight
layer1.2.bn2.weight
layer1.2.bn2.bias
layer1.2.conv3.weight
layer1.2.bn3.weight
layer1.2.bn3.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.conv3.weight
layer2.0.bn3.weight
layer2.0.bn3.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.we

In [10]:
for name, param in list(model.state_dict().items()):
    print(name)

conv1.weight
bn1.weight
bn1.bias
bn1.running_mean
bn1.running_var
bn1.num_batches_tracked
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.bn1.running_mean
layer1.0.bn1.running_var
layer1.0.bn1.num_batches_tracked
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.bn2.running_mean
layer1.0.bn2.running_var
layer1.0.bn2.num_batches_tracked
layer1.0.conv3.weight
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.bn3.running_mean
layer1.0.bn3.running_var
layer1.0.bn3.num_batches_tracked
layer1.0.downsample.0.weight
layer1.0.downsample.1.weight
layer1.0.downsample.1.bias
layer1.0.downsample.1.running_mean
layer1.0.downsample.1.running_var
layer1.0.downsample.1.num_batches_tracked
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.bn1.running_mean
layer1.1.bn1.running_var
layer1.1.bn1.num_batches_tracked
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.bn2.running_mean
layer1.1.bn2.running_var
layer1.1.bn2.num_batches_tr

In [2]:
class Identity(torch.nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

model = torchvision.models.resnet50(weights = None)
#model = SupCEResNet()
model_names = list()
for name, params in model.state_dict().items():
    model_names.append(name)
print(len(model_names))

320


In [3]:
model.fc = Identity()

In [4]:
loaded = torch.load('./supcon.pth', map_location = 'cpu')
loaded.keys()

dict_keys(['model', 'contrast', 'optimizer', 'epoch', 'model_ema'])

In [5]:
loaded_state_dict = loaded['model']
print(len(loaded_state_dict.keys()))


322


In [6]:
new_state_dict = dict()
for name, params in loaded_state_dict.items():
    name = name.replace('module.encoder.', '')
    new_state_dict[name] = params

print(len(new_state_dict))


322


In [7]:
print('loaded \ model')
for name in new_state_dict.keys():
    if name not in model_names:
        print(name)
    
print('*'*50)

print('model \ loaded')
for name in model_names:
    if name not in new_state_dict.keys():
        print(name)

loaded \ model
module.head.0.weight
module.head.0.bias
module.head.2.weight
module.head.2.bias
**************************************************
model \ loaded
fc.weight
fc.bias


In [8]:
state_dict = dict()
for name, params in new_state_dict.items():
    if not name.startswith('module.head'):
        state_dict[name] = params

print(len(state_dict))

318


In [9]:
model.load_state_dict(state_dict, strict = True)

<All keys matched successfully>

In [10]:
for name, param in model.named_parameters():
    if 'fc' in name:
        print(name)
    else:
        param.requires_grad = False

In [11]:
encoder = model

from classifier import Classifier
classifier = Classifier(10)
classifier

ModuleNotFoundError: No module named 'classifier'

In [ ]:
from trainer import Trainer
trainer = Trainer()

In [ ]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def multi_accuracy(output : torch.tensor, target : torch.tensor, S : int = 1):
    '''
    batch = mini batch or full batch
    output : (batch, class) : dosent matter after softmax or not
    target : (batch, )
    '''
    output = torch.softmax(output, dim = 1)
    pred = torch.argmax(output, dim = 1)
    pred = torch.tensor(pred.clone().detach() / S, dtype = int)

    total_corrects = sum(pred == target)

    return total_corrects / len(pred)

In [ ]:
optimizer = torch.optim.SGD(set(list(encoder.parameters())+ list(encoder.parameters())), lr=0.1, momentum=0.9, weight_decay=5e-4)
trainer.initialize(
    train_loader = train_loader,
        valid_loader = test_loader,
        test_loader = test_loader,
        encoder = encoder.cuda(),
        optimizer = optimizer,
        classifier = classifier.cuda(),
        lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
                optimizer, 
                milestones = [60, 110, 150, 180], 
                gamma=0.2),
        loss_function = torch.nn.CrossEntropyLoss().cuda(),
        accuracy_function = multi_accuracy)

In [7]:
import torchvision
import torch
data_set = torchvision.datasets.CIFAR10(root = './data', train=True,
                                        download=True, transform=None)

# Split the dataset into training and validation sets
train_len = int(len(data_set) * 0.5)
valid_len = len(data_set) - train_len
train_set, valid_set = torch.utils.data.random_split(data_set, [train_len, valid_len])


train_set.dataset.transforms = 1
valid_set.dataset.transforms

Files already downloaded and verified


1

1